## Preamble with imports

In [ ]:
from scipy import signal
from __future__ import print_function
import numpy as np
import sys 
import random
import time
from random import shuffle
import matplotlib
import matplotlib.pyplot as plt 
from matplotlib import colors
matplotlib.rcParams['figure.figsize'] = [12, 9]
%matplotlib inline

font = {'family' : 'sans',
        'weight' : 'regular',
        'size'   : 11}

matplotlib.rc('font', **font)


## Yeast Cell class

In [ ]:
#Yeast Cell Class

class yeast_cell:
    def __init__(self, live=1, gen=0, leaf=0,location=(0,0), gluconD=0,plate=1,id=1):
        self.location=location #coordinates, as tuples
        self.gluconD=gluconD #cells are glycolytic by default. Turning this on allows them to produce trehalose and diffuse it around
        self.C=0.5 #amount of C inside
        self.N=0.5 #amount of N inside
        self.id=1
        self.clock = 0 #keeps track of division time. 


## Diffusion Array

In [ ]:
DiffMat = np.array([[0,0.24,0],
                  [0.24,0.04,0.24],
                  [0,0.24,0]])

In [ ]:
def FillProb(locn):
    count=0
    cx=locn[0]
    cy=locn[1]
    cell_neigh = [(max(0,cx-1),cy),(cx,min(cy+1,gridleny-1)),(min(cx+1,gridlenx-1),cy),(cx,max(0,cy-1))]
    shuffle(cell_neigh)
    for locn in cell_neigh:
        if locn in LOC_STATE.keys():
            count+=1
    return count

In [ ]:
iter_len=750 #Give the system enough time to switch and grow

In [ ]:
'''
Default set of parameters (do not edit unless you've noted them down somewhere)

iter_len = 750 #No. of time steps to run the simulation

#MAIN PARAMETER MODELS AND INITIAL CONDITIONS
food=1.0 #how many units of food does each cell need to divide via glycolysis
GlucInit=0.0 #units of glucose available at each grid location at t=0
gridlenx=201 #grid dimension x
gridleny=201 #grid dimension y
trehalose=GlucInit*np.ones(shape=(gridlenx,gridleny)) #Trehalose plate 1 at t=0
aspartate=1e7*np.ones(shape=(gridlenx,gridleny))

ThreshLD = 0.0001 #Trehalose levels below which light cells can turn dark 
ProbLD = 0.0001 #probablity of light cells switching dark once threshold conditions are met

ThreshDL = 1.5 #Trehalose levels above which dark cells can turn light 
ProbDL = 0.5 #probablity of dark cells switching light once threshold conditions are met 

Cmax = 0.05 #maximum trehalose consumed by light cells per time step
AspU=4.0 #aspartate uptake multiplier
f=0.125 #fraction of Aspartate influx that contributes to Nitrogen reserves in dark cells
Y = 0.31 #A yield coefficient converting Asp to C
Pf = 0.049 #Fraction of internal C that is secreted as trehalose
MaxProduced = 0.12 #Maximum trehalose produced by dark cells per time step


ExN=4.0 #How much extra N do light cells need to meet N requirements for division.
Growth = 0.04 #probability of division per cell block per unit time once C and N reserves are met. 
'''

In [ ]:

#MAIN PARAMETER MODELS AND INITIAL CONDITIONS
food=1.0 #how many units of food does each cell need to divide via glycolysis
GlucInit=0.0 #units of glucose available at each grid location at t=0
gridlenx=201 #grid dimension x
gridleny=201 #grid dimension y
trehalose=GlucInit*np.ones(shape=(gridlenx,gridleny)) #Trehalose plate 1 at t=0
aspartate=1e7*np.ones(shape=(gridlenx,gridleny))

ThreshLD = 0.0001 #Trehalose levels below which light cells can turn dark 
ProbLD = 0.0001 #probablity of light cells switching dark once threshold conditions are met

ThreshDL = 1.5 #Trehalose levels above which dark cells can turn light 
ProbDL = 0.5 #probablity of dark cells switching light once threshold conditions are met 

Cmax = 0.05 #maximum trehalose consumed by light cells per time step
AspU=4.0 #aspartate uptake multiplier
f=0.125 #fraction of Aspartate influx that contributes to Nitrogen reserves in dark cells
Y = 0.31 #A yield coefficient converting Asp to C
Pf = 0.049 #Fraction of internal C that is secreted as trehalose
MaxProduced = 0.12 #Maximum trehalose produced by dark cells per time step


ExN=4.0 #How much extra N do light cells need to meet N requirements for division.
Growth = 0.04 #probability of division per cell block per unit time once C and N reserves are met.

In [ ]:
# Diagnostic variables
ActualMax=0
ActualSecreted = []
SwitchCount = 0 #counts switching
Switching = []
Switching.append(SwitchCount) #tracks switching
SwitchDist = [] #tracks switching events at a certain distance
SwitchTime = []
GlyDivTime = [] #keeps track of division time for light cells (make a histogram)
GluDivTime = [] #keeps track of division time for dark cells (make a histogram)

In [ ]:
#Initial Variables the ones which we tend to keep track of
GLUCON_FRAC=[]
POPU_TOTAL=[] #keeps track of total cell population
POPU_NUM=[] #tracks number 
GLUCON_X=[]
GLUCON_Y=[]
LOC=[] #all coordinates
COLONY_GRID = np.zeros(shape=(gridlenx,gridleny))
ColonyTime = np.zeros(shape=(iter_len+1,gridlenx,gridleny)) #store snapshots of the colony grid
ResourceTime = np.zeros(shape=(iter_len+1,gridlenx,gridleny)) #store snapshots of the trehalose grid
LOC_STATE={}


In [ ]:
#seed island with cells
GLUCON_COUNTER=0 #gluconeogenic cells
GStart = 0.999 #starting fraction of gluconeogenic cells
GLUCON_FRAC.append(GStart)
cx=int((len(trehalose[:-1])-1)/2)
cy=cx
originx=cx
originy=cy
cell_id=0
island = []
radius = 20
for x in range(gridlenx):
    for y in range(gridlenx):
        if (np.sqrt((cx-x)**2+(cy-y)**2))<=radius:
            island.append((x,y))

for locn in island: #starting island of cells
    glucons = random.sample(set(island),int(GStart*len(island))) #set up required number of gluconeogenic cells
    yseed=yeast_cell()
    yseed.id=cell_id
    cell_id+=1
    yseed.location=(locn[0],locn[1])
    LOC_STATE[locn] = 0 # 0 is glycolytic
    ColonyTime[0][locn]=1
    POPU_TOTAL.append(yseed)
    LOC.append(yseed.location)
    
    if locn in glucons:
        yseed.gluconD=1
        LOC_STATE[locn] = 1 # 1 is gluconeogenic
        ColonyTime[0][locn]=2
        GLUCON_COUNTER += 1

POPU_NUM.append(len(POPU_TOTAL))
tstart = time.time()

for time_iters in range(iter_len+1): #Number of fixed width time steps
    ResourceTime[time_iters] = trehalose
    trehalose = signal.convolve2d(trehalose,DiffMat,boundary='symm', mode='same') #diffusion step
    ColonyTime[time_iters] = ColonyTime[max(0,time_iters-1)]
    # initialise new generation cells
    nextgen=[]
    for a_cell in POPU_TOTAL: #sweep over the whole colony
        cx=a_cell.location[0]
        cy=a_cell.location[1]
        cell_neigh = [(max(0,cx-1),cy),(cx,min(cy+1,gridleny-1)),(min(cx+1,gridlenx-1),cy),(cx,max(0,cy-1))] # '+' shaped neighbourhood

        #Gluconeogenic (dark) cell block
        if a_cell.gluconD == 1: #check if the cell is Dark
            if random.random() <= ProbDL and trehalose[cx,cy] >= ThreshDL:
                a_cell.gluconD=0 #Switch to glycolytic (Light)
                LOC_STATE[(cx,cy)] = 0 #turn back to glycolytic state
                ColonyTime[time_iters][(cx,cy)] = 1
                SwitchCount += 1
                GLUCON_COUNTER -= 1
                SwitchDist.append(np.sqrt((cx-originx)**2+(cy-originy)**2))
                SwitchTime.append(time_iters)
 
            else:
                a_cell.N += AspU*f*Cmax
                a_cell.C += Y*AspU*(1.0-f)*Cmax
                a_cell.clock += 1
                Produced = min(Pf*a_cell.C,MaxProduced)
                if (a_cell.C-Produced) >= 0: 
                    a_cell.C -= Produced #remove secreted amount from internal C reserves
                    trehalose[cx,cy]=trehalose[cx,cy]+Produced #trying to couple the trehalose to the C reserves
                    ActualSecreted.append(Produced)
                                
                if a_cell.N >= food and a_cell.C >= food : #chance of dividing as a gluconeogenic (Dark) cell. 
                    if random.random()<=Growth:
                        empty_spot=[] # find an empty space to divide into
                        SpotScore = []
                        for locn in cell_neigh:
                            if locn not in LOC:
                                empty_spot.append(locn)
                                SpotScore.append(FillProb(locn))

                        if len(empty_spot)>0: #divide into available empty spot
                            (newglux,newgluy)=empty_spot[SpotScore.index(max(SpotScore))]
                            newcell=yeast_cell()
                            newcell.id=cell_id
                            newcell.gluconD = 1
                            cell_id+=1
                            newcell.C = a_cell.C/2.0
                            newcell.N = a_cell.N/2.0
                            a_cell.C = a_cell.C/2.0 #daughters get half
                            a_cell.N = a_cell.N/2.0 #daughters get half
                            GluDivTime.append(a_cell.clock)
                            a_cell.clock = 0 #reset division time clock
                            nextgen.append(newcell)
                            newcell.location=(newglux,newgluy)
                            LOC.append(newcell.location)
                            LOC_STATE[(newglux,newgluy)] = 1
                            ColonyTime[time_iters][(newglux,newgluy)] = 2
                            GLUCON_COUNTER+=1

        #Glycolytic (Light) cell
        elif a_cell.gluconD == 0:
            if random.random()<=ProbLD and trehalose[cx,cy] < ThreshLD:
                a_cell.gluconD=1 #switch to gluconeogenic (Dark)
                LOC_STATE[a_cell.location] = 1
                ColonyTime[time_iters][(cx,cy)]=2
                SwitchCount += 1 #count switching between states
                GLUCON_COUNTER+=1
                SwitchDist.append(np.sqrt((cx-originx)**2+(cy-originy)**2))
                SwitchTime.append(time_iters)
     
            else:
                a_cell.clock += 1
                a_cell.N += AspU*Cmax #Take up aspartate
                if trehalose[cx,cy] >0: #Take up trehalose
                    a_cell.C += min(Cmax,trehalose[cx,cy])
                    trehalose[cx,cy]=trehalose[cx,cy]-min(Cmax,trehalose[cx,cy])
                           
                if a_cell.C >= food and a_cell.N >= ExN*food:
                    if random.random() <=Growth:
                        empty_spot=[] # find an empty space to divide into
                        SpotScore=[]
                        for locn in cell_neigh:
                            if locn not in LOC:
                                empty_spot.append(locn)
                                SpotScore.append(FillProb(locn))
                        if len(empty_spot)>0:
                            (newx,newy)=empty_spot[SpotScore.index(max(SpotScore))]
                            newcell=yeast_cell()
                            newcell.id=cell_id
                            cell_id+=1
                            newcell.C = a_cell.C/2.0 #daughters get half
                            newcell.N = a_cell.N/2.0 #daughters get half
                            a_cell.C = a_cell.C/2.0 #daughters get half
                            a_cell.N = a_cell.N/2.0 #daughters get half
                            GlyDivTime.append(a_cell.clock)
                            a_cell.clock = 0 #reset division time clock
                            nextgen.append(newcell)
                            newcell.location=(newx,newy)
                            LOC_STATE[(newx,newy)] = 0
                            ColonyTime[time_iters][(newx,newy)] = 1
                            LOC.append(newcell.location)

                
    POPU_TOTAL=POPU_TOTAL+nextgen #update population by adding new generation
    POPU_NUM.append(len(POPU_TOTAL))
    GLUCON_FRAC.append(float(GLUCON_COUNTER)/len(POPU_TOTAL))
    Switching.append(float(SwitchCount)/float(POPU_NUM[-1]))

np.save('Colony_default',ColonyTime)
np.save('ResourceGrid',ResourceTime)

FINAL_GLUCON_CELLS=0
FINAL_LIVE_CELLS=0
for a_cell in POPU_TOTAL:
    if a_cell.gluconD==1:
        FINAL_GLUCON_CELLS+=1
        FINAL_LIVE_CELLS+=1
    elif a_cell.gluconD==0:
        FINAL_LIVE_CELLS+=1

print (GLUCON_COUNTER)
print (len(LOC_STATE.keys()) - len(POPU_TOTAL))
print ("Population: " + str(len(POPU_TOTAL)))

In [ ]:
plt.figure()
plt.hist(ActualSecreted,bins=50)
plt.grid(True)
plt.xlabel('Actual secreted amount')
figname="Secreted_trehalose_histogram"
plt.savefig(figname+'.svg',ext='svg',DPI=600)
plt.savefig(figname+'.png',ext='png',DPI=600)
plt.show()
print (np.mean(ActualSecreted))


In [ ]:
plt.figure()
plt.hist(GlyDivTime,normed=True,alpha=0.75,label='Light Cell Division time')
plt.hist(GluDivTime,normed=True,alpha=0.75,label='Dark Cell Division time')
plt.grid(True)
plt.legend(loc=0)
plt.xlabel('Division clock time')
plt.ylabel('Normalised division time frequency')
plt.title("Mean Division time ratio:"+str(np.mean(GluDivTime)/np.mean(GlyDivTime))[0:4])

ax = plt.gca()
ax.set_facecolor('#d3d3d3')
figname="Division_times"
plt.savefig(figname+'.svg',ext='svg',DPI=600)
# plt.savefig(figname+'.png',ext='png',DPI=600)